In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Point

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 定义自定义颜色映射
colors = ['#FF0000', '#FFFF00', '#00FF00']  # 红黄绿
n_bins = 100
cmap = LinearSegmentedColormap.from_list('custom_diverging', colors, N=n_bins)

# 定义模型类
class SolarNet(nn.Module):
    def __init__(self):
        super(SolarNet, self).__init__()
        # 编码器
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # 解码器
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up2 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.up3 = nn.ConvTranspose2d(32, 1, kernel_size=2, stride=2)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.relu(self.up1(x))
        x = torch.relu(self.up2(x))
        x = self.up3(x)
        return self.sigmoid(x)

try:
    # 1. 加载模型和数据
    model = SolarNet()
    model.load_state_dict(torch.load("../result/solar_model.pth", 
                                    weights_only=True))
    model.eval()

    # 2. 加载数据
    cloud_data = np.load("../data/processed/avg_cloud_thickness.npy")
    slope_data = np.load("../data/processed/slope_resampled.npy")

    # 3. 进行预测
    X = np.stack([cloud_data, slope_data], axis=0)
    X = torch.tensor(X, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        predictions = model(X).squeeze().numpy()

    # 4. 生成经纬度网格
    latitudes = np.linspace(1.2, 1.5, predictions.shape[0])
    longitudes = np.linspace(103.6, 104.1, predictions.shape[1])
    lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing='ij')

    # 使用百分位数作为阈值
    threshold = np.percentile(predictions, 80)  # 选取前20%的位置
    print(f"\n使用阈值（80百分位）：{threshold:.4f}")
    suitable_mask = predictions > threshold

    # 确保数组形状匹配
    assert predictions.shape == lat_grid.shape == lon_grid.shape, "数组形状不匹配"

    # 6. 提取合适位置的坐标和预测值
    suitable_locations = pd.DataFrame({
        'longitude': lon_grid[suitable_mask],
        'latitude': lat_grid[suitable_mask],
        'suitability_score': predictions[suitable_mask]
    })

    # 归一化适宜度分数
    min_score = predictions.min()
    max_score = predictions.max()
    suitable_locations['normalized_score'] = (suitable_locations['suitability_score'] - min_score) / (max_score - min_score)

    # 7. 保存结果
    suitable_locations.to_csv("D:/Project/ML/Singapore cloud project/result/suitable_locations.csv", 
                            index=False)

    # 8. 输出统计信息
    print("\n预测结果统计：")
    print(f"找到的适合位置数量：{len(suitable_locations)}")
    print(f"最小预测值：{predictions.min():.4f}")
    print(f"最大预测值：{predictions.max():.4f}")
    print(f"平均预测值：{predictions.mean():.4f}")
    print(f"预测值中位数：{np.median(predictions):.4f}")

    # 9. 可视化结果
    plt.figure(figsize=(20, 10))

    # 预测分布热图
    plt.subplot(131)
    im = plt.imshow(predictions, cmap=cmap)
    plt.colorbar(im, label='预测值')
    plt.title('预测值分布热图')

    # 适合位置散点图
    plt.subplot(132)
    scatter = plt.scatter(suitable_locations['longitude'], 
                         suitable_locations['latitude'], 
                         c=suitable_locations['normalized_score'],
                         cmap=cmap,
                         s=50,
                         alpha=0.6)
    plt.colorbar(scatter, label='归一化适宜度得分')
    plt.title('适合位置分布图')
    plt.xlabel('经度')
    plt.ylabel('纬度')

    # 在地图上标注前十个最适合的点
    plt.subplot(133)
    
    # 创建GeoDataFrame并选择前10个最适合的位置
    top_10_locations = suitable_locations.nlargest(10, 'suitability_score')
    geometry = [Point(xy) for xy in zip(top_10_locations['longitude'], top_10_locations['latitude'])]
    gdf = gpd.GeoDataFrame(top_10_locations, geometry=geometry, crs='EPSG:4326')
    
    # 转换为Web Mercator投影以匹配底图
    gdf = gdf.to_crs(epsg=3857)
    
    # 绘制底图和点
    ax = plt.gca()
    gdf.plot(ax=ax, color='red', markersize=100, zorder=2)
    
    # 添加底图
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    
    # 添加标签
    for idx, row in gdf.iterrows():
        plt.annotate(f'#{idx+1}\n得分: {row.suitability_score:.4f}', 
                    xy=(row.geometry.x, row.geometry.y),
                    xytext=(10, 10),
                    textcoords='offset points',
                    color='red',
                    fontsize=8,
                    bbox=dict(facecolor='white', alpha=0.7))
    
    plt.title('前十个最适合位置')
    ax.set_axis_off()

    # 保存图像
    plt.suptitle('新加坡太阳能电站选址分析结果', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.savefig("../result/evaluation_results.png", 
                dpi=300, bbox_inches='tight')
    plt.close()

    # 10. 区域统计分析
    print("\n区域分布统计：")
    suitable_locations['lon_region'] = pd.cut(suitable_locations['longitude'], 5)
    suitable_locations['lat_region'] = pd.cut(suitable_locations['latitude'], 5)
    region_stats = suitable_locations.groupby(['lon_region', 'lat_region']).size()
    print(region_stats)

    # 11. 输出前十个最佳位置的详细信息
    print("\n前十个最佳位置的详细信息：")
    print(top_10_locations[['longitude', 'latitude', 'suitability_score']].to_string(index=True))

except Exception as e:
    print(f"\n错误：{str(e)}")
    print("\n请检查以下几点：")
    print("1. 确保已安装必要的包：")
    print("   pip install geopandas contextily")
    print("2. 确保模型输出的predictions与经纬度网格大小一致")
    print("3. 检查cloud_data和slope_data的形状是否匹配")
    print("4. 确保所有文件路径正确")
    print("5. 检查网络连接（需要下载地图底图）")


使用阈值（80百分位）：0.0001

预测结果统计：
找到的适合位置数量：307
最小预测值：0.0000
最大预测值：0.0055
平均预测值：0.0002
预测值中位数：0.0000

区域分布统计：
lon_region        lat_region    
(103.599, 103.7]  (1.209, 1.268]    10
                  (1.268, 1.326]     5
                  (1.326, 1.384]    11
                  (1.384, 1.442]    12
                  (1.442, 1.5]      24
(103.7, 103.8]    (1.209, 1.268]     8
                  (1.268, 1.326]     5
                  (1.326, 1.384]    10
                  (1.384, 1.442]     9
                  (1.442, 1.5]      15
(103.8, 103.9]    (1.209, 1.268]    10
                  (1.268, 1.326]     6
                  (1.326, 1.384]    11
                  (1.384, 1.442]    10
                  (1.442, 1.5]      17
(103.9, 104.0]    (1.209, 1.268]    13
                  (1.268, 1.326]     9
                  (1.326, 1.384]    13
                  (1.384, 1.442]    14
                  (1.442, 1.5]      17
(104.0, 104.1]    (1.209, 1.268]    21
                  (1.268, 1.326]    14
    

C:\Users\13495\AppData\Local\Temp\ipykernel_15956\828030695.py:164: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  region_stats = suitable_locations.groupby(['lon_region', 'lat_region']).size()
